In [2]:
#%%-------------------------------------------------------------------------------------------
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch

import functions_IMU
import functions_generell
import functions_PRESSURE

#%% Daten einlesen für alle Teilnehmer-------------------------------------------------------------------------------------------
# liste der Teilneher
participants = ['ID_1_Dabisch_Samuel', 'ID_2_Pohl_Jannis', 'ID_3_Kleber_Christian',
                'ID_4_Schröter_Till', 'ID_5_Zaschke_Lenard', 'ID_6_Petroll_Finn', 'ID_7_Gruber_Julius']
# bedingunegn der Messungen
measurements = ['REAL_1', 'REAL_2', 'VR_1', 'VR_2']

# Alle Daten der Teilnehmer einlesen und verarbeiten
all_data = {}

for participant in participants:
    folder_path = os.path.join('data_final', participant)
    
    # Lade die 6 CSV-Dateien für den Teilnehmer
    real_1, real_2, vr_1, vr_2, mvc_beine, mvc_hals = functions_generell.load_csvs(folder_path)
    
    # Speichere die DataFrames im Dictionary
    all_data[participant] = {
        'REAL_1': real_1,
        'REAL_2': real_2,
        'VR_1': vr_1,
        'VR_2': vr_2,
        'MVC_Beine': mvc_beine,
        'MVC_Hals': mvc_hals
    }
    
    print(f"Daten für {participant} geladen")

print(f"\nAlle Daten geladen! {len(all_data)} Teilnehmer erfolgreich eingelesen.")

Daten für ID_1_Dabisch_Samuel geladen
Daten für ID_2_Pohl_Jannis geladen
Daten für ID_3_Kleber_Christian geladen
Daten für ID_4_Schröter_Till geladen
Daten für ID_5_Zaschke_Lenard geladen
Daten für ID_6_Petroll_Finn geladen
Daten für ID_7_Gruber_Julius geladen

Alle Daten geladen! 7 Teilnehmer erfolgreich eingelesen.


In [5]:
### takeofoff mit kniewinkel
def identify_jumps_by_knee(df, angle_col='RT Knee Flexion (deg)', min_jump_distance_sec=5.0, threshold_angle=60, buffer=0.75, fs=2000):
    """
    Identifiziert Sprünge sequentiell ohne Filterung:
    Sucht nach dem Peak > 60° und nimmt das ALLERERSTE lokale Minimum danach als Takeoff.
    """
    angle = df[angle_col].values
    time = df['time'].values
    
    jumps_list = []
    i = 0
    pause_points = int(min_jump_distance_sec * fs)
    search_window_points = int(1.5 * fs)

    while i < len(angle):
        # A. Suche nach dem nächsten Punkt über dem Schwellenwert (Ausholbewegung)
        if angle[i] >= threshold_angle:
            # 1. Den exakten Peak (maximale Beugung) im Umkreis finden
            peak_search_end = min(i + int(0.5 * fs), len(angle))
            peak_idx = i + np.argmax(angle[i:peak_search_end])
            
            # 2. Suche Takeoff: Das ERSTE lokale Minimum nach dem Peak
            takeoff_idx = peak_idx # Fallback
            
            # Wir laufen vom Peak vorwärts und suchen den ersten Punkt, 
            # an dem die Kurve nicht mehr weiter fällt.
            for j in range(peak_idx + 1, min(peak_idx + search_window_points, len(angle) - 1)):
                if angle[j] <= angle[j-1] and angle[j] <= angle[j+1]:
                    takeoff_idx = j
                    break
            else:
                # Falls kein lokales Minimum gefunden wurde, Backup: Absolutes Minimum
                search_end = min(peak_idx + search_window_points, len(angle))
                takeoff_idx = peak_idx + np.argmin(angle[peak_idx:search_end])

            # 3. Daten extrahieren
            t_absprung = time[takeoff_idx]
            start_analyse = max(0, t_absprung - buffer)
            
            jumps_list.append({
                'sprung nr.': len(jumps_list) + 1,
                'start_analyse': round(start_analyse, 4),
                't_absprung': round(t_absprung, 4)
            })
            
            # 4. Sperrfrist: 5 Sekunden nach dem Takeoff überspringen
            i = takeoff_idx + pause_points
            continue
        
        i += 1
            
    return jumps_list

def visualize_knee_jumps(df, jumps_list, participant="", measurement=""):
    """
    Visualisiert die Sprungerkennung basierend auf dem Kniewinkel.
    Korrektur: alpha in tick_params entfernt, da nicht unterstützt.
    """
    time = df['time'].values
    # Sicherstellen, dass die Spalte exakt so heißt
    knee_angle = df['RT Knee Flexion (deg)'].values
    total_force = df['LT Force (N)'] + df['RT Force (N)']
    
    fig, ax1 = plt.subplots(figsize=(15, 7))

    # Primäre Achse: Kniewinkel
    ax1.plot(time, knee_angle, color='blue', alpha=0.6, label='Kniewinkel (RT Flexion)')
    ax1.set_ylabel('Kniewinkel (deg)', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue') # alpha entfernt

    # # Sekundäre Achse für die Kraft
    # ax2 = ax1.twinx()
    # ax2.plot(time, total_force, color='black', alpha=0.2, label='Gesamtkraft (N)')
    # ax2.set_ylabel('Kraft (N)', color='gray') 
    # ax2.tick_params(axis='y', labelcolor='gray') # alpha entfernt

    labeled_buffer = False
    labeled_takeoff = False

    for j in jumps_list:
        # 1. Analyse-Fenster (Buffer bis Takeoff)
        # Beachte: Die Keys müssen 'start_analyse' und 't_absprung' sein (wie in deiner Funktion)
        ax1.axvspan(j['start_analyse'], j['t_absprung'], 
                    color='gray', alpha=0.15, 
                    label='Analyse-Fenster (Buffer)' if not labeled_buffer else "")
        labeled_buffer = True
        
        # 2. Vertikale Linie für Takeoff
        ax1.axvline(j['t_absprung'], color='red', linestyle='--', lw=2,
                    label='Takeoff (Knie-Extremum)' if not labeled_takeoff else "")
        labeled_takeoff = True
        
        # 3. Sprungnummer
        ax1.text(j['t_absprung'], max(knee_angle) * 0.95, f"S{j['sprung nr.']}", 
                 color='red', fontweight='bold', ha='right')

    plt.title(f"Knie-basierte Sprungerkennung: {participant} - {measurement} ({len(jumps_list)} Sprünge)")
    ax1.set_xlabel("Zeit (s)")
    
    # Legenden zusammenführen
    lines1, labels1 = ax1.get_legend_handles_labels()
    # lines2, labels2 = ax2.get_legend_handles_labels()
    ax1.legend(lines1 , labels1 , loc='upper right')
    
    ax1.grid(True, alpha=0.2)
    plt.tight_layout()
    plt.show()

output_folder = 'jump_analysis_results_knee'

# Erstelle Ausgabeverzeichnis, falls nicht vorhanden
os.makedirs(output_folder, exist_ok=True)

for participant in participants:
    print(f"\nVerarbeite {participant}...")
    
    all_jumps_data = []
    
    # Durchlaufe alle 4 Messungen
    for measurement in measurements:
        df = all_data[participant][measurement]
        
        # Identifiziere Sprünge in dieser Messung
        jumps = identify_jumps_by_knee(df,  min_jump_distance_sec = 5, buffer=1.0)
        #visualize_knee_jumps(df, jumps, participant, measurement)

        # Füge Measurement-Info zu jedem Sprung hinzu
        for jump in jumps:
            jump['messung'] = measurement
            all_jumps_data.append(jump)
        
        print(f"  {measurement}: {len(jumps)} Sprünge gefunden")
    
    # Erstelle DataFrame aus allen Sprüngen
    jumps_df = pd.DataFrame(all_jumps_data)
    
    # Speichere als CSV
    output_file = os.path.join(output_folder, f'{participant}_jumps.csv')
    jumps_df.to_csv(output_file, index=False)
    
    print(f"  ✓ Gesamt: {len(all_jumps_data)} Sprünge")
    print(f"  ✓ CSV gespeichert: {output_file}")

print(f"\n{'='*80}")
print(f"Verarbeitung abgeschlossen!")
print(f"Alle CSVs wurden im Ordner '{output_folder}' gespeichert.")
print(f"{'='*80}")



Verarbeite ID_1_Dabisch_Samuel...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_1_Dabisch_Samuel_jumps.csv

Verarbeite ID_2_Pohl_Jannis...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 5 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 1 Sprünge gefunden
  ✓ Gesamt: 18 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_2_Pohl_Jannis_jumps.csv

Verarbeite ID_3_Kleber_Christian...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_3_Kleber_Christian_jumps.csv

Verarbeite ID_4_Schröter_Till...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 5 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 23 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_knee\ID_4_Schröter_Till_jumps.csv

Verarbeite 

In [8]:
### parameter aus Kraftdaten extrahieren
def identify_jumps(df, flight_threshold=50, min_flight_sec=0.2):
    """
    Identifiziert Sprünge über einen Kraft-Schwellenwert und berechnet die Sprunghöhe.
    
    Parameters:
    -----------
    flight_threshold : float
        Kraftwert (N), unter dem eine Flugphase erkannt wird (Default: 50N).
    min_flight_sec : float
        Mindestdauer in der Luft, um als Sprung zu gelten.
    """
    total_force = (df['LT Force (N)'] + df['RT Force (N)']).values
    time = df['time'].values
    g = 9.81 # Erdbeschleunigung
    
    # 1. Flugphasen-Maske
    in_air = total_force < flight_threshold
    
    # 2. Detektion von Statuswechseln
    diff = np.diff(in_air.astype(int))
    takeoff_indices = np.where(diff == 1)[0]
    landing_indices = np.where(diff == -1)[0]
    
    # Korrektur der Indizes (Start mit Take-off, Ende mit Landung)
    if len(takeoff_indices) > 0 and len(landing_indices) > 0:
        if landing_indices[0] < takeoff_indices[0]:
            landing_indices = landing_indices[1:]
        takeoff_indices = takeoff_indices[:len(landing_indices)]

    jumps_list = []
    
    # 3. Flugdaten extrahieren und Höhe berechnen
    for i in range(len(takeoff_indices)):
        idx_off = takeoff_indices[i]
        idx_on = landing_indices[i]
        
        t_off = time[idx_off]
        t_on = time[idx_on]
        flug_dauer = t_on - t_off
        
        # Filter für plausible Sprünge
        if flug_dauer > min_flight_sec:
            # Berechnung der Sprunghöhe in Metern
            # Formel: h = 1/8 * g * t^2
            sprunghoehe_m = (1/8) * g * (flug_dauer**2)
            sprunghoehe_cm = sprunghoehe_m * 100
            
            jump_dict = {
                'sprung nr.': len(jumps_list) + 1,
                't_absprung': round(t_off, 4),
                't_landung': round(t_on, 4),
                'flugzeit_s': round(flug_dauer, 4),
                'sprunghoehe_cm': round(sprunghoehe_cm, 2)
            }
            jumps_list.append(jump_dict)
            
    return jumps_list

def visualize_jumps(df, jumps_list, participant, measurement):
    # Kraft berechnen
    total_force = df['LT Force (N)'] + df['RT Force (N)']
    time = df['time'].values
    
    plt.figure(figsize=(15, 7))
    plt.plot(time, total_force, color='black', alpha=0.3, label='Rohdaten Kraft')
    
    # Hilfsvariablen für die Legende
    labeled_flight_zone = False
    labeled_events = False
    
    for j in jumps_list:
        # 1. Die Flugphase (t_absprung bis t_landung) markieren
        plt.axvspan(j['t_absprung'], j['t_landung'], 
                    color='orange', alpha=0.2, 
                    label='Flugphase' if not labeled_flight_zone else "")
        labeled_flight_zone = True
        
        # 2. Vertikale Linien für Take-off und Landing
        plt.axvline(j['t_absprung'], color='green', linestyle='--', alpha=0.6, 
                    label='Take-off' if not labeled_events else "")
        plt.axvline(j['t_landung'], color='red', linestyle='--', alpha=0.6, 
                    label='Landing' if not labeled_events else "")
        labeled_events = True
        
        # 3. Text-Position (Mitte der Flugphase)
        text_pos_x = j['t_absprung'] + (j['flugzeit_s'] / 2)
        
        # 4. Sprungnummer und Sprunghöhe anzeigen
        # Wir platzieren den Text etwas über der Kraftkurve
        y_pos = max(total_force) * 0.9
        plt.text(text_pos_x, y_pos, f"Sprung {j['sprung nr.']}\n{j['sprunghoehe_cm']} cm", 
                 horizontalalignment='center', fontweight='bold', fontsize=10,
                 bbox=dict(facecolor='white', alpha=0.8, edgecolor='orange', boxstyle='round,pad=0.5'))
        
        # 5. Flugzeit darunter anzeigen
        plt.text(text_pos_x, y_pos * 0.82, f"t: {j['flugzeit_s']}s", 
                 horizontalalignment='center', fontsize=9, color='black', alpha=0.7)

    # Titel und Labels
    title_str = f"Sprungdetektion: {participant} - {measurement} | {len(jumps_list)} Sprünge gefunden"
    plt.title(title_str, fontsize=14, pad=15)
    plt.xlabel("Zeit (s)")
    plt.ylabel("Gesamtkraft (N)")
    
    # Verzeichnis erstellen (falls nicht vorhanden)
    save_dir = 'Pictures_Test/Sprungzyklen_Kraft'
    os.makedirs(save_dir, exist_ok=True)
    
    plt.legend(loc='upper right')
    plt.grid(True, alpha=0.2)
    plt.tight_layout()
    
    # Speichern
    #file_name = f'{participant}_{measurement}_jump_analysis.png'
    #plt.savefig(os.path.join(save_dir, file_name), dpi=300)

    plt.show()


output_folder = 'jump_analysis_results_Pressure'

# Erstelle Ausgabeverzeichnis, falls nicht vorhanden
os.makedirs(output_folder, exist_ok=True)

for participant in participants:
    print(f"\nVerarbeite {participant}...")
    
    all_jumps_data = []
    
    # Durchlaufe alle 4 Messungen
    for measurement in measurements:
        df = all_data[participant][measurement]
        
        # Identifiziere Sprünge in dieser Messung
        jumps = identify_jumps(df, flight_threshold=155, min_flight_sec=0.2)
        #visualize_jumps(df, jumps, participant, measurement)

        # Füge Measurement-Info zu jedem Sprung hinzu
        for jump in jumps:
            jump['messung'] = measurement
            all_jumps_data.append(jump)
        
        print(f"  {measurement}: {len(jumps)} Sprünge gefunden")
    
    # Erstelle DataFrame aus allen Sprüngen
    jumps_df = pd.DataFrame(all_jumps_data)
    
    # Speichere als CSV
    output_file = os.path.join(output_folder, f'{participant}_jumps.csv')
    jumps_df.to_csv(output_file, index=False)
    
    print(f"  ✓ Gesamt: {len(all_jumps_data)} Sprünge")
    print(f"  ✓ CSV gespeichert: {output_file}")

print(f"\n{'='*80}")
print(f"Verarbeitung abgeschlossen!")
print(f"Alle CSVs wurden im Ordner '{output_folder}' gespeichert.")
print(f"{'='*80}")



Verarbeite ID_1_Dabisch_Samuel...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_1_Dabisch_Samuel_jumps.csv

Verarbeite ID_2_Pohl_Jannis...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 5 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 2 Sprünge gefunden
  ✓ Gesamt: 19 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_2_Pohl_Jannis_jumps.csv

Verarbeite ID_3_Kleber_Christian...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_3_Kleber_Christian_jumps.csv

Verarbeite ID_4_Schröter_Till...
  REAL_1: 6 Sprünge gefunden
  REAL_2: 6 Sprünge gefunden
  VR_1: 6 Sprünge gefunden
  VR_2: 6 Sprünge gefunden
  ✓ Gesamt: 24 Sprünge
  ✓ CSV gespeichert: jump_analysis_results_Pressure\ID_4_Schröter_Till_jumps.